In [10]:
import json

with open('./task4.json') as f:
    data = json.load(f)


In [11]:
# If use Together API
from together import Together

your_model_name = 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free'
your_api_key = '574bf95f6b33074e60f284c2c6316e72c5ef8226ce8ce4650dacca4d51ce1050'
client = Together(api_key=your_api_key)

In [12]:

import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Inicializar listas para almacenar statements y generations
statements = []
generations = []

# Procesar cada statement y generar su correspondiente respuesta
for statement in data['Statements']:
    country = statement['country']
    system_prompt = f"""
    Assume you are the representative of {country} in UNSC, given the content of a UNSC draft resolution, generate a statement that you would make in the meeting.
    """
    user_prompt = f"""
    Draft resolution: {data['Content']}
    Your statement:
    """
    
    response = client.chat.completions.create(
        model=your_model_name,
        temperature=0.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # Guardar el statement original y la generación
    statements.append(statement['statement'])
    generations.append(response.choices[0].message.content)

# Calcular similitudes
tf_cosine_scores = []
for i in range(len(statements)):
    tfidf = TfidfVectorizer().fit_transform([statements[i], generations[i]])
    score = cosine_similarity(tfidf[0], tfidf[1])[0][0]
    tf_cosine_scores.append(score)

print('Average Cosine Similarity (TF-IDF):', sum(tf_cosine_scores) / len(tf_cosine_scores))

Average Cosine Similarity (TF-IDF): 0.6675293071521021


In [13]:
# Importar las bibliotecas necesarias
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Inicializar las listas
statements = []
generations = []

# Ya que data es un diccionario y no una lista, accedemos directamente a 'Statements'
for statement in data['Statements']:
    statements.append(statement['statement'])
    # Verificar si 'generation' existe en el statement
    if 'generation' in statement:
        generations.append(statement['generation'])
    else:
        # Si no existe, generamos uno nuevo usando el modelo
        country = statement['country']
        system_prompt = f"""
        Assume you are the representative of {country} in UNSC, given the content of a UNSC draft resolution, generate a statement that you would make in the meeting.
        """
        user_prompt = f"""
        Draft resolution: {data['Content']}
        Your statement:
        """
        
        response = client.chat.completions.create(
            model=your_model_name,
            temperature=0.0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        generation = response.choices[0].message.content
        generations.append(generation)

# Calcular similitudes usando TF-IDF
tf_cosine_scores = []
for i in range(len(statements)):
    tfidf = TfidfVectorizer().fit_transform([statements[i], generations[i]])
    score = cosine_similarity(tfidf[0], tfidf[1])[0][0]
    tf_cosine_scores.append(score)

print('Average Cosine Similarity (TF-IDF):', sum(tf_cosine_scores) / len(tf_cosine_scores))

Average Cosine Similarity (TF-IDF): 0.7272845170779837


In [14]:
# average ROUGE
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

rouge_scores = []
for i in range(len(statements)):
    scores = scorer.score(statements[i], generations[i])
    rouge_scores.append(scores['rougeL'].fmeasure)

print('Average ROUGE-L:', sum(rouge_scores) / len(rouge_scores))

Average ROUGE-L: 0.2200956937799043


In [15]:
# average Jaccard similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

jaccard_scores = []
vectorizer = CountVectorizer(binary=True)

for i in range(len(statements)):
    X = vectorizer.fit_transform([statements[i], generations[i]]).toarray()
    jaccard_scores.append(jaccard_score(X[0], X[1]))

print('Average Jaccard similarity:', sum(jaccard_scores) / len(jaccard_scores))

Average Jaccard similarity: 0.21428571428571427


In [16]:
# average Cosing Similarity (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tf_cosine_scores = []
for i in range(len(statements)):
    tfidf = TfidfVectorizer().fit_transform([statements[i], generations[i]])
    tf_cosine_scores.append(cosine_similarity(tfidf[0], tfidf[1])[0][0])

print('Average Cosine Similarity (TF-IDF):', sum(tf_cosine_scores) / len(tf_cosine_scores))

Average Cosine Similarity (TF-IDF): 0.7272845170779837


In [17]:
# Imprimir resumen final de todas las métricas
print('\nResumen Final de Métricas de Similitud:')
print('-' * 40)
print(f'TF-IDF Cosine Similarity: {sum(tf_cosine_scores) / len(tf_cosine_scores):.4f}')
print(f'ROUGE-L Score:            {sum(rouge_scores) / len(rouge_scores):.4f}')
print(f'Jaccard Similarity:       {sum(jaccard_scores) / len(jaccard_scores):.4f}')
print('-' * 40)

# Opcionalmente, podemos guardar los resultados
results = {
    'tf_idf': sum(tf_cosine_scores) / len(tf_cosine_scores),
    'rouge_l': sum(rouge_scores) / len(rouge_scores),
    'jaccard': sum(jaccard_scores) / len(jaccard_scores)
}

# Guardar los resultados en un archivo JSON
with open('similarity_results.json', 'w') as f:
    json.dump(results, f, indent=2)



Resumen Final de Métricas de Similitud:
----------------------------------------
TF-IDF Cosine Similarity: 0.7273
ROUGE-L Score:            0.2201
Jaccard Similarity:       0.2143
----------------------------------------
